<a href="https://colab.research.google.com/github/vinayk19/Assignment/blob/master/CNN_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
import torch.nn.functional as F
import time

In [0]:
train_data = datasets.MNIST(root = "MNIST/processed/training.pt", train = True, transform=transforms.ToTensor(), download= True)
test_data = datasets.MNIST(root = "MNIST/processed/test.pt", train = False, transform=transforms.ToTensor(), download = True)

In [0]:
train_loader = DataLoader(dataset = train_data, batch_size= 64, shuffle= True)
test_loader = DataLoader(dataset = test_data, batch_size= 64, shuffle= False)

In [36]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("device is ", device)

device is  cuda


In [0]:
class mnist(torch.nn.Module):
  def __init__(self):
    super(mnist, self).__init__()
    self.conv1 = torch.nn.Conv2d(1, 10 , kernel_size= 5, stride= 1, padding= 0)
    self.conv2 = torch.nn.Conv2d(10, 20, kernel_size= 5, stride= 1, padding= 0)
    self.maxpool = torch.nn.MaxPool2d(kernel_size=2)
    self.linear = torch.nn.Linear(320,10) #how 320 comes up

  def forward(self, data):
    in_size = data.size(0)
    x = F.relu(self.maxpool(self.conv1(data)))
    x = F.relu(self.maxpool(self.conv2(x)))  
    x = x.view(in_size, -1) 
    x = self.linear(x)
    out = F.log_softmax(x)
    return out

needs to undedrstand mistake below

In [0]:
# class mnist(torch.nn.Module):
#   def __init__(self):
#     super(mnist, self).__init__()
#     self.conv1 = torch.nn.Conv2d(1, 10 , kernel_size= 5, stride= 1, padding= 0)
#     self.conv2 = torch.nn.Conv2d(10, 20, kernel_size= 5, stride= 1, padding= 0) ()
#     self.maxpool = torch.nn.MaxPool2d(kernel_size=2)
#     self.linear = torch.nn.Linear(320,10) #how 320 comes up

#   def forward(self, data):
#     in_size = data.size(0)
#     x = torch.nn.ReLU(self.maxpool(self.conv1(data))) # this relu is not givenng tensort output
#     x = torch.nn.ReLU(self.maxpool(self.conv2(x)))  
#     x = torch.view(in_size, -1)  # x.view
#     x = self.linear(x)
#     out = torch.nn.functional.log_softmax(x)
#     return out

In [38]:
model = mnist()
model.to(device)

mnist(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (linear): Linear(in_features=320, out_features=10, bias=True)
)

Loss

In [0]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=.01, momentum=.4)

trainig

In [0]:
# def train(epoch):
#   model.train()
#   for Epo in range(epoch):
#     for batch_idx, (images, labels) in enumerate(train_loader):
#       images, labels = images.to(device), labels.to(device)
#       targets = model(images)
#       optimizer.zero_grad()
#       loss = criterian(targets, labels)
#       loss.backward()
#       optimizer.step()
#       if batch_idx % 500  == 0:
#         # print('Train Epoch: {} | Batch Status: {}/{} ({:.0f}% | Loss: {:.6f}', format(
#         #     epoch, batch_idx*len(data), len(train_loader.dataset), 100. *batch_idx / len(train_loader), loss.item()
#         # ))
#         print('Train Epoch: {} | Batch Status: {}/{} ({:.0f}%) | Loss: {:.6f}'.format(
#                 epoch, batch_idx * len(data), len(train_loader.dataset),
#                 100. * batch_idx / len(train_loader), loss.item()))



In [0]:

def train(epoch):
  model.train()
  for Epoch in range(5):
    for batch_idx, (data, target) in enumerate(train_loader):
      data, target = data.to(device), target.to(device)
      optimizer.zero_grad()
      logit = model(data)
      loss = criterion(logit, target)
      loss.backward()
      optimizer.step()
      if batch_idx % 500  == 0:
        # print('Train Epoch: {} | Batch Status: {}/{} ({:.0f}% | Loss: {:.6f}', format(
        #     epoch, batch_idx*len(data), len(train_loader.dataset), 100. *batch_idx / len(train_loader), loss.item()
        # ))
        print('Train Epoch: {} | Batch Status: {}/{} ({:.0f}%) | Loss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [0]:
def test():
  model.eval()  # what is meaning of it ?
  with torch.no_grad(): # its not compulsary but it will save some merory
    test_loss =0
    correct = 0
    for data, target in test_loader: # enumeration is not required as this is a single batch
      data, target = data.to(device), target.to(device)
      output = model(data)  # output.shape = 64,10
      #sum up the match loss
      test_loss += criterion(output, target).item()
      #get the index of the max value
      pred = output.data.max(1, keepdim=True)[1]  #output.data is ouput data/value ? what is max function https://www.journaldev.com/39463/pytorch-torch-max
      #1 is axis: 0 is row (ie colum wise max:> 1, 10, and 1 is colum ie row wise max)
      # provides max data and and its index
      correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print(f'===========================\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} '
            f'({100. * correct / len(test_loader.dataset):.0f}%)')
    # print(f'====================\n test set : Average loss: {test_loss: .4f}, Accuracy: {correct}/{len(test_loader.dataset)} ' 
            # f'({100. * correct/len(test_loader.dataset):. 0f}%)')
            
  #         ===========================
  # Test set: Average loss: 0.0016, Accuracy: 9787/10000 (98%)
  # Testing timr: 0m 25s

In [43]:
if __name__ == '__main__':
  since = time.time()
  for epoch in range(1,2):
    epoch_start = time.time()
    train(epoch)
    m, s = divmod(time.time() - epoch_start, 60)
    print(f'Training timr: {m:.0f}m {s:.0f}s')
    test()
    m, s = divmod(time.time() - epoch_start, 60)
    print(f'Testing timr: {m:.0f}m {s:.0f}s')
  m, s = divmod(time.time() - since, 60)
  print(f'Total Time: {m:.0f}m {s:.0f}s\nModel was trained on {device}!')

Train Epoch: 1 | Batch Status: 0/60000 (0%) | Loss: 2.304240


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  from ipykernel import kernelapp as app


Train Epoch: 1 | Batch Status: 32000/60000 (53%) | Loss: 0.240961
Train Epoch: 1 | Batch Status: 0/60000 (0%) | Loss: 0.229553
Train Epoch: 1 | Batch Status: 32000/60000 (53%) | Loss: 0.073775
Train Epoch: 1 | Batch Status: 0/60000 (0%) | Loss: 0.169380
Train Epoch: 1 | Batch Status: 32000/60000 (53%) | Loss: 0.051236
Train Epoch: 1 | Batch Status: 0/60000 (0%) | Loss: 0.075967
Train Epoch: 1 | Batch Status: 32000/60000 (53%) | Loss: 0.181949
Train Epoch: 1 | Batch Status: 0/60000 (0%) | Loss: 0.182886
Train Epoch: 1 | Batch Status: 32000/60000 (53%) | Loss: 0.065528
Training timr: 0m 24s
Test set: Average loss: 0.0011, Accuracy: 9789/10000 (98%)
Testing timr: 0m 25s
Total Time: 0m 25s
Model was trained on cuda!
